Conhecendo os dados (o que são? de onde vêm?), refletindo sobre como lidar com eles e efetivamente transformando-os!

## Agenda
        1 - Explorando os Dados
            Identificando os desafios
                1.1 ['Localização']
                1.2 ['Preço']
                1.3 ['Tamanho (m2)']
                1.4 ['Número de quartos'] e ['vagas para estacionar']
                1.5 ['Número de banheiros']
        2 - Manipulação dos dados (Limpeza - Data Cleaning)
            Eliminando coluna ['Unnamed: 0'] e linhas sem preço ("Sob consulta")
            2.1 ['X_tamanho_em_m2']
            2.2 ['y_preço']
            2.3 ['X_número_de_quartos'], ['X_vagas_para_estacionar'] 
                ['X_número_de_banheiros']
            2.4 Salvando Alterações

In [1]:
import pandas as pd

In [2]:
data_frame = pd.read_csv('Dados Apartamentos.csv')
data_frame.head()

,Unnamed: 0,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar
0,0,"Rua Eurico de Souza Leão, Cordeiro",\n Sob consulta\n,55 - 73 m²,1 - 3,1,1 - 2
1,1,"Rua Marquês de Tamandaré, Poço",\n Sob consulta\n,111 - 209 m²,3,3,2
2,2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1
3,3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2
4,4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1


## 1 - Explorando os Dados
Um roteiro básico para entender o que temos em mãos, e ter uma ideia de como lidar com eles.

In [3]:
data_frame.shape

(6768, 7)

    Grande parte das colunas numéricas estão no formato object (str, dict ou list...). 
    Teremos que converter futuramente, mas pelo que vemos na tabela acima, não será um
    processo tão simples assim.

In [6]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6768 entries, 0 to 6767
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             6768 non-null   int64 
 1   Localização            6768 non-null   object
 2   Preço                  6768 non-null   object
 3   Tamanho (m2)           6768 non-null   object
 4   Número de quartos      6768 non-null   object
 5   Número de banheiros    6768 non-null   int64 
 6   vagas para estacionar  6768 non-null   object
dtypes: int64(2), object(5)
memory usage: 370.2+ KB


    Menos mal que não há NaN, mas em .head() podemos perceber que há valores
    em ['Preço'] que a priori talvez tenhamos que eliminar da análise.

In [5]:
data_frame.isna().sum()

Unnamed: 0               0
Localização              0
Preço                    0
Tamanho (m2)             0
Número de quartos        0
Número de banheiros      0
vagas para estacionar    0
dtype: int64

In [8]:
data_frame.columns

Index(['Unnamed: 0', 'Localização', 'Preço', 'Tamanho (m2)',
       'Número de quartos', 'Número de banheiros', 'vagas para estacionar'],
      dtype='object')

## Identificando os desafios.
Agora, vamos inspecionar cada ['coluna'] individualmente para saber os desafios que estão por vir. E já começar a racioncinar como resolvê-los.

### 1.1 ['Localização']

In [32]:
# 1110 endereços diferentes \o\|o|/o/

len(data_frame['Localização'].unique())

1110

Estava pensando em separar os dados por bairro. Aqui vejo que será uma tarefa bem mais trabalhosa do que pensei. Já que na linha 3, por exemplo, temos o nome do bairro no meio da string "Avenida **Boa Viagem**, Recife", enquanto que nas linhas 6766 e 6767 ele está no final (o que facilita nosso trabalho) "Rua Professor Júlio Ferreira de Melo, **Boa Viagem**".

Há técnicas que nos auxiliam nesse tipo de problema (**regular expressions - Regex**), mas ainda não sou muito familiarizado com elas. Lidar com isso vai ser um verdadeiro challenge pra mim.

hehe

In [33]:
data_frame['Localização'].head()

0    Rua Eurico de Souza Leão, Cordeiro
1        Rua Marquês de Tamandaré, Poço
2          Rua General Polidoro, Várzea
3            Avenida Boa Viagem, Recife
4        Avenida Santos Dumont, Aflitos
Name: Localização, dtype: object

In [34]:
data_frame['Localização'].tail()

6763                 Travessa Maria Carolina, Boa Viagem
6764       Rua Professor Otávio de Freitas, Encruzilhada
6765             Rua Jerônimo de Albuquerque, Casa Forte
6766    Rua Professor Júlio Ferreira de Melo, Boa Viagem
6767                     Rua Solidônio Leite, Boa Viagem
Name: Localização, dtype: object

### 1.2 ['Preço']

Pela forma que fiz o web scraping, bastantes preços vieram com informações adicionais. Não esperava por isso e poderia apenas ignorar o conteúdo extra, mas talvez possa até ser útil. Se não souber como tirar proveito deles, simplesmente
os deletarei.

In [37]:
len(data_frame['Preço'].unique())

23

In [36]:
data_frame['Preço'].unique()

array(['\n    Sob consulta\n  ', '\n    R$ 262.900\n  ',
       '\n    R$ 1.700.000\n  ',
       '\n    R$ 450.000\n  \n      condomínio\n      R$ 460\n      IPTU\n      R$ 1.600',
       '\n    R$ 2.500.000\n  \n      condomínio\n      R$ 1.600',
       '\n    R$ 499.000\n  \n      condomínio\n      R$ 650\n      IPTU\n      R$ 1.500',
       '\n    R$ 350.000\n  \n      condomínio\n      R$ 600\n      IPTU\n      R$ 1.200',
       '\n    R$ 480.000\n  \n      condomínio\n      R$ 445\n      IPTU\n      R$ 180',
       '\n    R$ 280.000\n  \n      condomínio\n      R$ 400',
       '\n    R$ 750.000\n  \n      condomínio\n      R$ 1.760',
       '\n    R$ 320.000\n  \n      condomínio\n      R$ 340\n      IPTU\n      R$ 1.300',
       '\n    R$ 245.000\n  \n      condomínio\n      R$ 450',
       '\n    R$ 4.500\n  ',
       '\n    R$ 680.000\n  \n      condomínio\n      R$ 780\n      IPTU\n      R$ 2.922',
       '\n    R$ 450.000\n  ',
       '\n    R$ 250.000\n  \n      condomínio\n

### 1.3 ['Tamanho (m2)']
Acredito que a única problemática aqui seja fazer a média dos _ranges_ de dados como "55 - 73 m²" ou "111 - 209 m²"

In [38]:
len(data_frame['Tamanho (m2)'].unique())

23

In [40]:
data_frame['Tamanho (m2)'].unique()

array(['        55 - 73 m²', '        111 - 209 m²', '        69 m²',
       '        230 m²', '        54 m²', '        204 m²',
       '        70 m²', '        74 m²', '        89 m²',
       '        110 m²', '        186 m²', '        51 m²',
       '        66 m²', '        600 m²', '        103 m²',
       '        180 m²', '        55 m²', '        104 m²',
       '        87 m²', '        142 m²', '        82 m²',
       '        145 m²', '        86 m²'], dtype=object)

### 1.4 ['Número de quartos'] e ['vagas para estacionar']

Por ter esse tipo de dado em _range_ "1 - 3" e "1 - 2" podemos tratar todos os dados como uma "classe" (mesmo os outros não sendo um intervalo/range, ainda assim eles seriam outras classes) ou tirar a "média", se bem que 1.5 quartos não parece ser uma quantidade normal de quartos em uma casa. No momento, não tenho certeza de como tratar esse ponto.

In [45]:
data_frame['Número de quartos'].unique()

array(['        1 - 3 ', '        3 ', '        2 ', '        4 '],
      dtype=object)

In [48]:
data_frame['vagas para estacionar'].unique()

array(['        1 - 2 ', '        2 ', '        1 ', '        3 ',
       '        5 '], dtype=object)

### 1.5 ['Número de banheiros']
O queridinho... idealmente ideal... 0 defeitos. Aprecie (\*-\* )

In [46]:
data_frame['Número de banheiros'].unique()

array([1, 3, 5, 2, 4], dtype=int64)

## 2 - Manipulação dos dados (Limpeza - Data Cleaning)
Continuando a explorar os dados, podemos começar a fazer alterações de acordo com nosso objetivo. Para isso, podemos começar a **limpar os dados**, transformando-os de maneira a conseguirmos utilizarmos depois.

    ps: Ao criar as novas colunas utilizei ['X_nome'] para se referir às features e ['y_nome'] ao label
        (para se assimilar à linguagem padrão machine learning --> X_train, y_train, X_test, y_test)
    
    E também mative bastante o .head() para acompanhar as mudanças.

In [14]:
data_frame.head(2)

,Unnamed: 0,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar
0,0,"Rua Eurico de Souza Leão, Cordeiro",\n Sob consulta\n,55 - 73 m²,1 - 3,1,1 - 2
1,1,"Rua Marquês de Tamandaré, Poço",\n Sob consulta\n,111 - 209 m²,3,3,2


**Primeiro, vamos deletar a coluna ['Unnamed: 0']**. Ela é desnecessária.

In [15]:
data_frame.drop(columns='Unnamed: 0', inplace=True)
data_frame.head()

,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar
0,"Rua Eurico de Souza Leão, Cordeiro",\n Sob consulta\n,55 - 73 m²,1 - 3,1,1 - 2
1,"Rua Marquês de Tamandaré, Poço",\n Sob consulta\n,111 - 209 m²,3,3,2
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1


Agora **vamos eliminar os preços "Sob consulta"**, já que ainda não foram definidos. 
    
      Poderíamos fazer um tipo de agrupamento por localização e utilizar as outras variáveis
      (já "limpas") como features para aplicar um algoritmo de ML que pudesse estimar esses 
      valores. Sinto que há conhecimentos específicos da área (que não sei) que talvez 
      precisasse considerar ao fazer isso. Sem contar que é um procedimento que não tenho
      prática, então prefiro eliminar tais valores (até porque eles representam aproximadamente 
      8% dos dados totais, ou seja, é uma perda relativamente pequena).

In [16]:
data_frame.Preço[data_frame.Preço == '\n    Sob consulta\n  '].count()

564

In [19]:
data_frame.Preço[data_frame.Preço == '\n    Sob consulta\n  '].count() / data_frame.shape[0]

0.08333333333333333

In [95]:
new_data_frame = data_frame[data_frame.Preço != '\n    Sob consulta\n  ']
new_data_frame.head()

,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1


## 2.1 ['X_tamanho_em_m2']

Como o algoritmo de machine learning faz cálculos com as features, precisamos ter certeza que elas sejam números.

Coincidentemente ou não, ao eliminarmos as linhas em que os valores ['Preço'] não estavam definidos, também perdemos os dados em formato de intervalo/range como "55 - 73 m²"  ou  "111 - 209 m²". Logo, precisamos apenas separaros números das strings
"m²"

**lambda** rapidamente resolve essa tarefa.

In [69]:
new_data_frame['Tamanho (m2)'].unique()

array(['        69 m²', '        230 m²', '        54 m²',
       '        204 m²', '        70 m²', '        74 m²',
       '        89 m²', '        110 m²', '        186 m²',
       '        51 m²', '        66 m²', '        600 m²',
       '        103 m²', '        180 m²', '        55 m²',
       '        104 m²', '        87 m²', '        142 m²',
       '        82 m²', '        145 m²', '        86 m²'], dtype=object)

In [96]:
# Criaremos outra coluna para armazenar apenas os números (já no formato int)

new_data_frame['X_tamanho_em_m2'] = new_data_frame['Tamanho (m2)'].apply(lambda x: int(x.split(' ')[8]))
new_data_frame.head()

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70


### [O que é “A value is trying to be set on a copy of a slice from a dataframe”](https://paulovasconcellos.com.br/o-que-%C3%A9-a-value-is-trying-to-be-set-on-a-copy-of-a-slice-from-a-dataframe-e85f744d8be1)

## 2.2 ['y_preço']
Criar uma coluna apenas com o preço de compra do apartamento

In [98]:
new_data_frame['Preço'].unique()

array(['\n    R$ 262.900\n  ', '\n    R$ 1.700.000\n  ',
       '\n    R$ 450.000\n  \n      condomínio\n      R$ 460\n      IPTU\n      R$ 1.600',
       '\n    R$ 2.500.000\n  \n      condomínio\n      R$ 1.600',
       '\n    R$ 499.000\n  \n      condomínio\n      R$ 650\n      IPTU\n      R$ 1.500',
       '\n    R$ 350.000\n  \n      condomínio\n      R$ 600\n      IPTU\n      R$ 1.200',
       '\n    R$ 480.000\n  \n      condomínio\n      R$ 445\n      IPTU\n      R$ 180',
       '\n    R$ 280.000\n  \n      condomínio\n      R$ 400',
       '\n    R$ 750.000\n  \n      condomínio\n      R$ 1.760',
       '\n    R$ 320.000\n  \n      condomínio\n      R$ 340\n      IPTU\n      R$ 1.300',
       '\n    R$ 245.000\n  \n      condomínio\n      R$ 450',
       '\n    R$ 4.500\n  ',
       '\n    R$ 680.000\n  \n      condomínio\n      R$ 780\n      IPTU\n      R$ 2.922',
       '\n    R$ 450.000\n  ',
       '\n    R$ 250.000\n  \n      condomínio\n      R$ 700',
       '\n    R$ 3

In [162]:
new_data_frame['y_preço'] = new_data_frame['Preço'].apply(lambda x: x.split(' ')[5])
new_data_frame.head()

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2,y_preço
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69,262.900\n
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230,1.700.000\n
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54,450.000\n
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204,2.500.000\n
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70,499.000\n


In [163]:
new_data_frame['y_preço'] = new_data_frame['y_preço'].str[:-1]
new_data_frame.head()

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2,y_preço
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69,262.900
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230,1.700.000
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54,450.000
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204,2.500.000
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70,499.000


**Não é possível tranformar as strings em número, por haver mais de um ponto '.'. Então substituirei os pontos por nada ""**

In [164]:
new_data_frame['y_preço'] = new_data_frame['y_preço'].apply(lambda x: x.replace('.','' ))
new_data_frame.head()

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2,y_preço
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69,262900
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230,1700000
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54,450000
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204,2500000
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70,499000


In [166]:
new_data_frame['y_preço'] = pd.to_numeric(new_data_frame['y_preço'])
new_data_frame.head()

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2,y_preço
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69,262900
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230,1700000
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54,450000
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204,2500000
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70,499000


## 2.3 ['X_número_de_quartos'], ['X_vagas_para_estacionar'] 
Note que, como em ['tamanho'], os intervalos "1 - 2" se foram. Nesse caso, só precisamos converter as 'strings' para números.

In [167]:
new_data_frame['Número de quartos'].unique()

array(['        2 ', '        4 ', '        3 '], dtype=object)

In [168]:
new_data_frame['vagas para estacionar'].unique()

array(['        1 ', '        2 ', '        3 ', '        5 '],
      dtype=object)

In [169]:
new_data_frame['X_número_de_quartos'] = pd.to_numeric(new_data_frame['Número de quartos'])
new_data_frame['X_vagas_para_estacionar'] = pd.to_numeric(new_data_frame['vagas para estacionar'])
new_data_frame.head()

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2,y_preço,X_número_de_quartos,X_vagas_para_estacionar
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69,262900,2,1
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230,1700000,4,2
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54,450000,2,1
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204,2500000,4,3
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70,499000,2,1


### ['X_número_de_banheiros']

**Criando a coluna com o nome padronizado. Sim, eu poderia ter simplesmente mudado o nome das colunas originais utilizando .replace(" ","_") mas não fiz. Nenhum motivo especial, apenas não fiz kkkk.**

In [175]:
new_data_frame['X_número_de_banheiros'] = new_data_frame['Número de banheiros']

C:\Users\pc\.conda\envs\tenserflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [176]:
new_data_frame.head()

,Localização,Preço,Tamanho (m2),Número de quartos,Número de banheiros,vagas para estacionar,X_tamanho_em_m2,y_preço,X_número_de_quartos,X_vagas_para_estacionar,X_número_de_banheiros
2,"Rua General Polidoro, Várzea",\n R$ 262.900\n,69 m²,2,1,1,69,262900,2,1,1
3,"Avenida Boa Viagem, Recife",\n R$ 1.700.000\n,230 m²,4,5,2,230,1700000,4,2,5
4,"Avenida Santos Dumont, Aflitos",\n R$ 450.000\n \n condomínio\n ...,54 m²,2,2,1,54,450000,2,1,2
5,"Rua Alameda das Hortências, Imbiribeira",\n R$ 2.500.000\n \n condomínio\n ...,204 m²,4,5,3,204,2500000,4,3,5
6,"Rua Cônego Romeu, Boa Viagem",\n R$ 499.000\n \n condomínio\n ...,70 m²,2,2,1,70,499000,2,1,2


## 2.4 Salvando Alterações
Por questões pessoais (tempo) e para não precisar rodar todo o notebook. Salvarei o progresso até agora em um arquivo .csv e posteriormente só precisarei carregá-lo e continuar o _**feature engineering**_.

In [179]:
df = new_data_frame[['Localização','X_tamanho_em_m2','X_número_de_quartos','X_vagas_para_estacionar','X_número_de_banheiros','y_preço']]
df.head()

,Localização,X_tamanho_em_m2,X_número_de_quartos,X_vagas_para_estacionar,X_número_de_banheiros,y_preço
2,"Rua General Polidoro, Várzea",69,2,1,1,262900
3,"Avenida Boa Viagem, Recife",230,4,2,5,1700000
4,"Avenida Santos Dumont, Aflitos",54,2,1,2,450000
5,"Rua Alameda das Hortências, Imbiribeira",204,4,3,5,2500000
6,"Rua Cônego Romeu, Boa Viagem",70,2,1,2,499000


In [180]:
df.shape

(6204, 6)

In [183]:
df.to_csv("Dados Pré-limpos.csv", index=False)

**That's it!**

**Phase 2 - Exploratory Data Analysis Complete!**

Na próxima fase, finalizarei o preprocessamento dos dados ['Localização'] e farei mais análises, desta vez com um olhar voltado para preparação do modelo (já que aqui o objetivo era análisar os dados para preparar sua limpeza e torná-los _usáveis_ ).